In [1]:
# Install dependencies as needed:
# pip install kagglehub[pandas-datasets]
import kagglehub
from kagglehub import KaggleDatasetAdapter

# Set the path to the file you'd like to load
file_path = "vgsales new.csv"

# Load the latest version
file = kagglehub.load_dataset(
  KaggleDatasetAdapter.PANDAS,
  "willianoliveiragibin/video-game-sales-analyze",
  file_path,
  # Provide any additional arguments like 
  # sql_query or pandas_kwargs. See the 
  # documenation for more information:
  # https://github.com/Kaggle/kagglehub/blob/main/README.md#kaggledatasetadapterpandas
)



/tmp/ipykernel_13/662714986.py:10: DeprecationWarning: load_dataset is deprecated and will be removed in a future version.
  file = kagglehub.load_dataset(


# Check Data And Drop Down Null Values 

In [2]:
file.shape

(16598, 11)

In [3]:
import pandas as pd 
file =  pd.DataFrame(file)

In [4]:
file.value_counts()
file.isna().sum()

Rank              0
Name              0
Platform          0
Year            271
Genre             0
Publisher        58
NA_Sales          0
EU_Sales          0
JP_Sales          0
Other_Sales       0
Global_Sales      0
dtype: int64

In [5]:
file =  file.dropna()

In [6]:
file.head()

,Rank,Name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales
0,259,Asteroids,2600,1980.0,Shooter,Atari,4.00,0.26,0.0,0.05,4.31
1,545,Missile Command,2600,1980.0,Shooter,Atari,2.56,0.17,0.0,0.03,2.76
2,1768,Kaboom!,2600,1980.0,Misc,Activision,1.07,0.07,0.0,0.01,1.15
3,1971,Defender,2600,1980.0,Misc,Atari,0.99,0.05,0.0,0.01,1.05
4,2671,Boxing,2600,1980.0,Fighting,Activision,0.72,0.04,0.0,0.01,0.77


# lets check sales  across the Platform 

In [7]:
import plotly.express as px
file.value_counts()

Rank   Name                             Platform  Year    Genre     Publisher                     NA_Sales  EU_Sales  JP_Sales  Other_Sales  Global_Sales
1      Wii Sports                       Wii       2006.0  Sports    Nintendo                      41.49     29.02     3.77      8.46         82.74           1
11062  Beowulf: The Game                PSP       2007.0  Action    Ubisoft                       0.08      0.00      0.00      0.01         0.09            1
11049  Castlevania Dracula X            SNES      1995.0  Platform  Konami Digital Entertainment  0.03      0.01      0.05      0.00         0.09            1
11050  Golden Nugget 64                 N64       1998.0  Misc      Electronic Arts               0.07      0.02      0.00      0.00         0.09            1
11051  Robotron 64                      N64       1998.0  Shooter   GT Interactive                0.07      0.02      0.00      0.00         0.09            1
                                                   

In [8]:
fig =  px.bar(data_frame=file,y='Platform',x='Global_Sales',orientation='h',category_orders={'Platform':file.groupby(['Platform']).agg({'Global_Sales':'sum'}).reset_index().sort_values(by='Global_Sales')['Platform'].to_list()})
fig.show(renderer='iframe_connected')
# from below graph we have highest sales for all platforms in ps2 
# lets check it time series 

In [9]:
fig =  px.line(data_frame=file,x='Year',y='Global_Sales',color='Platform')
#we have alot of platforms and its hard to compare , lets get only highest 10 platforms by value 
fig.show(renderer='iframe_connected')

In [10]:
#here we list top 10 
file.groupby(['Platform']).agg({'Global_Sales':'sum'}).sort_values(by='Global_Sales',ascending=False).head(10)


,Global_Sales
Platform,
PS2,1233.46
X360,969.60
PS3,949.35
Wii,909.81
DS,818.91
PS,727.39
GBA,305.62
PSP,291.71
PS4,278.10


In [11]:
file.groupby(['Platform']).agg({'Global_Sales':'sum'}).sort_values(by='Global_Sales',ascending=False).head(10).reset_index()['Platform'].to_list()

['PS2', 'X360', 'PS3', 'Wii', 'DS', 'PS', 'GBA', 'PSP', 'PS4', 'PC']

In [12]:
fig = px.line(data_frame=file[file['Platform'].isin(file.groupby(['Platform']).agg({'Global_Sales':'sum'}).sort_values(by='Global_Sales',ascending=False).head(10).reset_index()['Platform'].to_list())],x='Year',y='Global_Sales',color='Platform')
fig.show(renderer='iframe_connected')
#lets check tis high value of wii 

In [13]:
fig = px.box(data_frame=file[file['Platform'].isin(file.groupby(['Platform']).agg({'Global_Sales':'sum'}).sort_values(by='Global_Sales',ascending=False).head(10).reset_index()['Platform'].to_list())],y='Global_Sales',color='Platform',height=600)
fig.show(renderer='iframe_connected')
# as shown below we have outlier For Wii for 82 Million 
#lets go deeper to check what this come from 

In [14]:
file.loc[(file.Platform == 'Wii'   ) & (file.Global_Sales >82) ]
#from below table we have Nintendo with over 82.74 sales , we think that data is accurate as Na_sales and Eu sales is not that soo far . 
# over the web Nintendo is deleveopled as new video games generation so it may be have that impact , what ever lets go with other platforms 

,Rank,Name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales
6113,1,Wii Sports,Wii,2006.0,Sports,Nintendo,41.49,29.02,3.77,8.46,82.74


In [15]:
fig =  px.bar(data_frame=file[file['Platform'].isin(file.groupby(['Platform']).agg({'Global_Sales':'sum'}).sort_values(by='Global_Sales',ascending=False).head(10).reset_index()['Platform'].to_list())],x='Year',y='Global_Sales',color='Platform',barmode='overlay',height=600)
fig.show(renderer='iframe_connected')
# now we have more clear vision , ps have the most sales before the 2000 , from 2000 to 2005  we have competeion between ps2 and GPA , from 2005  to 2009 we can say Wii have the most sales maybe according to the next video games technology for sports games , from 2010 to 2014 
# x 360  and ps3 have the biggest comptetion , after 2014 ps4 have the king situation . 

# Rank and Sales Coorelation


In [16]:
# lest check the rank and sales realtion . 

fig =  px.scatter(data_frame=file[file['Platform'].isin(file.groupby(['Platform']).agg({'Global_Sales':'sum'}).sort_values(by='Global_Sales',ascending=False).head(10).reset_index()['Platform'].to_list())],y='Global_Sales',x='Rank')
fig.show(renderer='iframe_connected')
# no relation between rank and total sales 

# Sales Across Regions

In [17]:
#lets check sales accross areas 
file.head()

,Rank,Name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales
0,259,Asteroids,2600,1980.0,Shooter,Atari,4.00,0.26,0.0,0.05,4.31
1,545,Missile Command,2600,1980.0,Shooter,Atari,2.56,0.17,0.0,0.03,2.76
2,1768,Kaboom!,2600,1980.0,Misc,Activision,1.07,0.07,0.0,0.01,1.15
3,1971,Defender,2600,1980.0,Misc,Atari,0.99,0.05,0.0,0.01,1.05
4,2671,Boxing,2600,1980.0,Fighting,Activision,0.72,0.04,0.0,0.01,0.77


In [18]:
newfile =  pd.melt(file,id_vars=['Platform','Year'],value_vars=['NA_Sales','EU_Sales','JP_Sales','Other_Sales'],value_name='sales',var_name='Region')


In [19]:
fig =  px.imshow(file[['NA_Sales','EU_Sales','JP_Sales','Other_Sales']].corr(),width=1000)
fig.show(renderer='iframe_connected')
# Eu Sales And NA Sales Have Got Good Correlation lets check with scatter 

# Genres Analysis 

In [20]:
#lets go the next step for genre Analysis 
fig =  px.line(data_frame=file,x='Year',y='Global_Sales',color='Genre')
fig.show(renderer='iframe_connected')
# Okay We Can See greate jump in shoorter games at 1984 at that was age of simulation using shooting tools , then was the age of plaotform games like super mario or somting we can deep in next step to check in this areas , then we have sport games in 2005 and 2006 the was goldedn 
# age of fifa and pes across all platforms even PC 
